### Portions of this notebook adapted from https://github.com/rouseguy/DeepLearning-NLP/blob/master/notebooks/4.%20Recurrent%20Neural%20Networks.ipynb

In [1]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'F:\NeuralNetwork\word2vec\GoogleNews-vectors-negative300.bin', binary=True)


C:\Users\Stephen\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import keras
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras import backend as K
from keras.engine import Layer
from keras.models import Sequential
from keras.layers import *
from keras import *

Using TensorFlow backend.


In [3]:
from sklearn.cross_validation import train_test_split

C:\Users\Stephen\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import numpy as np

In [5]:
from string import *
reg_vocab = [word for word in model.vocab if all(letter in printable for letter in word)]

In [6]:
char_list = printable


In [7]:
char_indices = dict((c, i) for i, c in enumerate(char_list))
indices_char = dict((i, c) for i, c in enumerate(char_list))

In [8]:
MAX_LENGTH = 0
i = 0
MAX_INDEX = 0
for i,n in enumerate(reg_vocab):
    
    if len(n) > MAX_LENGTH:
        MAX_LENGTH = len(n)
        MAX_INDEX = i
        
print(MAX_LENGTH,MAX_INDEX)

98 7349


In [9]:
def word_to_char_seq(word):
    word_chars = list(word)
    word_chars_indices = list(map(lambda char: char_indices[char], word_chars))
    #return np.array(word_chars_indices)
    return sequence.pad_sequences([word_chars_indices], maxlen=MAX_LENGTH, padding='post')[0]

In [10]:
def vectorize_words(words, verbose=False):
    vwords =[]
    for name in words:
        x = [char_indices[c] for c in name]
        if verbose:
            print(x)
        vx = np.eye(len(char_indices))[x]  #https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.eye.html
        if verbose:
            print(vx[0:12])
        vwords.append(vx)
    return vwords

In [11]:
y = np.array([model.wv[word] for word in reg_vocab])
del model

C:\Users\Stephen\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [12]:
X = np.array([word_to_char_seq(word) for word in reg_vocab], dtype='uint8')

In [13]:
X_train, y_train, X_test, y_test = train_test_split(X, y)
del X
del y

In [14]:
X_train[0]

array([42, 53, 36, 48, 37, 47, 44, 49, 42, 88, 47, 10, 75, 88, 42, 27, 10,
       22, 11, 21, 18, 23, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=uint8)

In [15]:
X_train.shape

(2226641, 98)

In [16]:
### Adapted from here: https://nhanitvn.wordpress.com/2016/09/27/a-keras-layer-for-one-hot-encoding/

### I had no idea you could create your own layers!!!

class OneHot(Layer):
    '''Turn positive integers (indexes) into dense dummy vectors of fixed size
    eg. [[0], [1], [2]] -&gt; [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
 
    This layer can only be used as the first layer in a model.
 
    # Example
 
    ```python
        model = Sequential()
        model.add(OneHot(3))
        # now: model.output_shape == (None, 1, 3)
 
    # Arguments
        input_dim: int &gt; 0. Size of the vocabulary, ie.
 
    # Input shape
        2D tensor with shape: `(nb_samples, sequence_length)`.
 
    # Output shape
        3D tensor with shape: `(nb_samples, sequence_length, input_dim)`.
    '''
    def __init__(self, input_dim, input_length=None, **kwargs):
        self.input_dim = input_dim
        self.input_length = input_length
        kwargs['input_shape'] = (self.input_length,)
        kwargs['input_dtype'] = 'int32'
        super(OneHot, self).__init__(**kwargs)
        #self.get_output_shape_for(100, input_length=200)
 
    def get_output_shape_for(self, input_shape):
        if not self.input_length:
            input_length = input_shape[1]
        else:
            input_length = self.input_length
        print(input_shape, input_length, self.input_dim)
        return (input_length, input_shape[0], self.input_dim)
 
    def call(self, x, mask=None):
        return K.one_hot(x, self.input_dim)
 
    def get_config(self):
        config = {'input_dim': self.input_dim,
                  'input_length': self.input_length}
        base_config = super(OneHot, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [17]:
num_classes = len(printable)
k_model = Sequential()
#k_model.add(OneHot(len(printable)))
#k_model.add(Input(shape=(98,), dtype='uint8'))
#k_model.add(Lambda(K.one_hot, arguments={'num_classes': 100}, input_shape=(98,), output_shape=(98,100)))
# k_model.add(Lambda(to_categorical(100)))(input_shape=(1, 98))
onehot_weights = np.eye(num_classes)
k_model.add(Embedding(num_classes, num_classes,
                      trainable=False,
                      embeddings_initializer=keras.initializers.Identity(gain=1.0),
                      input_length=98,
                      #weights=[onehot_weights],
                      name='onehot'))
# k_model.add(Conv1D(filters=16, kernel_size=5, strides=1, padding='valid'))
# k_model.add(MaxPooling1D(pool_size=2))
# k_model.add(Flatten())
# k_model.add(Embedding(1,300))
k_model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [18]:
k_model.fit(X_train, y_train,
                    batch_size=100,
                    epochs=200,
                    verbose=1,
                    validation_split=0.1)

ValueError: Error when checking target: expected onehot to have 3 dimensions, but got array with shape (742214, 98)

In [ ]:
len(printable)

In [ ]:
print(X_train[0])

In [ ]:
X_train

In [ ]:
y_train